In [ ]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder , FunctionTransformer
from sklearn.impute import SimpleImputer
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback


In [ ]:
train_data = pd.read_csv("/home/adham/kaggle competitions/house-prices-advanced-regression-techniques/train.csv")
test_data = pd.read_csv("/home/adham/kaggle competitions/house-prices-advanced-regression-techniques/test.csv")
IDes = test_data['Id']
train_data.info()
IDes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [ ]:
cat = []
num_flo = []
num_int=[]
for col in train_data.columns:
  if train_data[col].dtype == 'object' and train_data[col].isnull().sum() < 0.5 * train_data.shape[0]:
    cat.append(col)

  elif train_data[col].dtype == 'int64':
    num_int.append(col)

  elif train_data[col].dtype == 'float64':
    num_flo.append(col)

num_int.remove('SalePrice')
num_int.remove('Id')

y_train = train_data['SalePrice']


train_data= train_data[cat + num_flo + num_int]
test_data= test_data[cat + num_flo + num_int]
num_flo



['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [ ]:
train_data[cat] = train_data[cat].fillna("NA")
test_data[cat]= test_data[cat].fillna("NA")
train_data[num_flo] = train_data[num_flo].fillna(0.0)
test_data[num_flo] = test_data[num_flo].fillna(0.0)




In [ ]:
sc = StandardScaler()
train_data[num_int + num_flo] = sc.fit_transform(train_data[num_int + num_flo])
test_data[num_int + num_flo] = sc.fit_transform(test_data[num_int + num_flo])

leng = len(train_data)

train_data = pd.concat([train_data,test_data])
train_data = pd.get_dummies(train_data)

test_data =  train_data[leng:]

train_data = train_data[:leng]

# test_data = pd.get_dummies(test_data)


In [ ]:
X_train,X_test, y_train,y_test = train_test_split(train_data,y_train, test_size = 0.20, random_state=42)
X_train

,LotFrontage,MasVnrArea,GarageYrBlt,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,...,SaleType_NA,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
254,0.357167,-0.570750,0.194602,-0.872563,-0.212153,-0.795151,0.381743,-0.472560,-1.350194,1.049169,...,0,0,0,1,0,0,0,0,1,0
1066,0.039729,-0.570750,0.273977,0.073375,-0.268578,-0.071836,1.280685,0.719786,0.442586,-0.973018,...,0,0,0,1,0,0,0,0,1,0
638,0.270593,-0.570750,-4.120324,-0.636078,-0.174369,-0.795151,1.280685,-2.029235,-1.689368,-0.973018,...,0,0,0,1,0,0,0,0,1,0
799,0.068587,0.824062,0.154915,-0.163109,-0.332419,-0.795151,1.280685,-1.134975,-1.689368,0.274948,...,0,0,0,1,0,0,0,0,1,0
380,-0.219993,-0.570750,0.121842,-0.163109,-0.552908,-0.795151,0.381743,-1.565545,-1.689368,-0.494887,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,0.588031,-0.570750,0.302641,-0.872563,-0.120249,-0.071836,-0.517200,1.150356,1.024029,-0.920380,...,0,0,0,1,0,0,0,0,1,0
1130,0.212877,-0.570750,0.247519,-0.163109,-0.271885,-1.518467,-2.315085,-1.433062,-1.689368,0.391191,...,0,0,0,1,0,0,0,0,1,0
1294,0.068587,-0.570750,0.194602,-0.872563,-0.235003,-0.795151,1.280685,-0.538802,0.248772,-0.606743,...,0,0,0,1,0,0,0,0,1,0
860,-0.075703,-0.570750,0.124046,-0.163109,-0.288121,0.651479,2.179628,-1.764269,0.636400,-0.973018,...,0,0,0,1,0,0,0,0,1,0


In [ ]:
pip install optuna

In [ ]:
import lightgbm as lgb
import optuna
from sklearn.model_selection import cross_val_score

def lgb_objective(trial):
    params = {
        "n_jobs": -1,
        'n_iter': 5,
        'verbosity': -1,

        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.1, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 20, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'learning_rate': trial.suggest_float('learning_rate', 0.02, 0.1, log=True),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-3, 20.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-3, 20.0),
        'num_leaves': trial.suggest_int('num_leaves', 4, 512),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 4, 512),
    }

    model  = lgb.LGBMRegressor(**params)
    scores = cross_val_score(
        model, X_train, y_train, cv=5, scoring='neg_mean_squared_error'
    )

    return np.mean(scores)

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(lgb_objective, n_trials=200)

best_params = study.best_params


[I 2024-02-20 20:50:50,939] A new study created in memory with name: no-name-718381e7-857a-45d2-b960-aff40235e930
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/usr/local/lib/python3.10/dist-pac

In [ ]:
best_params

{'colsample_bytree': 0.32556608256240677,
 'colsample_bynode': 0.4781735648094336,
 'n_estimators': 966,
 'max_depth': 17,
 'learning_rate': 0.09925018297648526,
 'lambda_l1': 1.3922996059492152,
 'lambda_l2': 0.004692567253543167,
 'num_leaves': 256,
 'min_data_in_leaf': 17}

In [ ]:
lgbr = lgb.LGBMRegressor(**best_params)
lgbr.fit(X_train,y_train)


[LightGBM] [Warning] lambda_l2 is set=0.004692567253543167, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.004692567253543167
[LightGBM] [Warning] min_data_in_leaf is set=17, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=17
[LightGBM] [Warning] lambda_l1 is set=1.3922996059492152, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.3922996059492152
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] lambda_l2 is set=0.004692567253543167, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.004692567253543167
[LightGBM] [Warning] min_data_in_leaf is set=17, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=17
[LightGBM] [Warning] lambda_l1 is set=1.3922996059492152, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.3922996059492152
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001236 seconds.
You can set `force_row_wise=tr

LGBMRegressor(colsample_bynode=0.4781735648094336,
              colsample_bytree=0.32556608256240677,
              lambda_l1=1.3922996059492152, lambda_l2=0.004692567253543167,
              learning_rate=0.09925018297648526, max_depth=17,
              min_data_in_leaf=17, n_estimators=966, num_leaves=256)

In [ ]:
y_pred = lgbr.score(X_test,y_test)
y_pred

[LightGBM] [Warning] lambda_l2 is set=0.004692567253543167, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.004692567253543167
[LightGBM] [Warning] min_data_in_leaf is set=17, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=17
[LightGBM] [Warning] lambda_l1 is set=1.3922996059492152, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.3922996059492152


0.8925980588509054

In [ ]:
y_pred = lgbr.predict(test_data)



# s = set(test_data)-set(train_data)
# s
# test_data['SalePrice']
# test_data['SalePrice'] = y_pred.astype(int)
IDes = pd.DataFrame(IDes)

IDes['SalePrice'] = pd.DataFrame(y_pred.astype(int))
IDes.head()
IDes.to_csv("submission.csv" , index=False)

[LightGBM] [Warning] lambda_l2 is set=0.004692567253543167, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.004692567253543167
[LightGBM] [Warning] min_data_in_leaf is set=17, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=17
[LightGBM] [Warning] lambda_l1 is set=1.3922996059492152, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.3922996059492152
